In [54]:
from qdrant_client import QdrantClient, models
client = QdrantClient("http://localhost:6333")



In [55]:
client

In [56]:
import json

data_path = r"C:\Users\m.salama\Desktop\my-workspace\llm-zc\week_0\documents.json"
documents_raw = json.load(open(data_path, "r", encoding="utf-8"))

In [57]:
for d in documents_raw:
    print(d["course"])

for d in documents_raw:
    print(d["documents"][0].keys())
    break

data-engineering-zoomcamp
machine-learning-zoomcamp
mlops-zoomcamp
dict_keys(['text', 'section', 'question'])


In [58]:
from fastembed import TextEmbedding
mdls = TextEmbedding.list_supported_models()

print(len(mdls))
for m in mdls:
    print(m["model"])

30
BAAI/bge-base-en
BAAI/bge-base-en-v1.5
BAAI/bge-large-en-v1.5
BAAI/bge-small-en
BAAI/bge-small-en-v1.5
BAAI/bge-small-zh-v1.5
mixedbread-ai/mxbai-embed-large-v1
snowflake/snowflake-arctic-embed-xs
snowflake/snowflake-arctic-embed-s
snowflake/snowflake-arctic-embed-m
snowflake/snowflake-arctic-embed-m-long
snowflake/snowflake-arctic-embed-l
jinaai/jina-clip-v1
Qdrant/clip-ViT-B-32-text
sentence-transformers/all-MiniLM-L6-v2
jinaai/jina-embeddings-v2-base-en
jinaai/jina-embeddings-v2-small-en
jinaai/jina-embeddings-v2-base-de
jinaai/jina-embeddings-v2-base-code
jinaai/jina-embeddings-v2-base-zh
jinaai/jina-embeddings-v2-base-es
thenlper/gte-base
thenlper/gte-large
nomic-ai/nomic-embed-text-v1.5
nomic-ai/nomic-embed-text-v1.5-Q
nomic-ai/nomic-embed-text-v1
sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2
sentence-transformers/paraphrase-multilingual-mpnet-base-v2
intfloat/multilingual-e5-large
jinaai/jina-embeddings-v3


In [59]:
EMBEDDING_DIMENSIONALITY = 512

for model in mdls:
    if model["dim"] == EMBEDDING_DIMENSIONALITY:
        print(model["model"])

BAAI/bge-small-zh-v1.5
Qdrant/clip-ViT-B-32-text
jinaai/jina-embeddings-v2-small-en


In [60]:
model_handle = "jinaai/jina-embeddings-v2-small-en"

collection_name = "zoomcamp-rag"

# Create the collection with specified vector parameters
client.create_collection(
    collection_name=collection_name,
    vectors_config=models.VectorParams(
        size=EMBEDDING_DIMENSIONALITY,  # Dimensionality of the vectors
        distance=models.Distance.COSINE  # Distance metric for similarity search
    )
)


UnexpectedResponse: Unexpected Response: 409 (Conflict)
Raw response content:
b'{"status":{"error":"Wrong input: Collection `zoomcamp-rag` already exists!"},"time":0.00367294}'

In [ ]:
points = []
id = 0

for course in documents_raw:
    for doc in course['documents']:

        point = models.PointStruct(
            id=id,
            vector=models.Document(text=doc['text'], model=model_handle), #embed text locally with "jinaai/jina-embeddings-v2-small-en" from FastEmbed
            payload={
                "text": doc['text'],
                "section": doc['section'],
                "course": course['course']
            } #save all needed metadata fields
        )
        points.append(point)

        id += 1


In [61]:
client.upsert(
    collection_name=collection_name,
    points=points
)

UpdateResult(operation_id=3, status=<UpdateStatus.COMPLETED: 'completed'>)

In [62]:
def search(query, limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle 
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [63]:
import random

course = random.choice(documents_raw)
course_piece = random.choice(course['documents'])
print(json.dumps(course_piece, indent=2))

{
  "text": "Initially, I did not assume there was a model2. I copied the original model1.bin and dv.bin. Then when I tried to load using\nCOPY [\"model2.bin\", \"dv.bin\", \"./\"]\nthen I got the error above in MINGW64 (git bash) on Windows.\nThe temporary solution I found was to use\nCOPY [\"*\", \"./\"]\nwhich I assume combines all the files from the original docker image and the files in your working directory.\nAdded by Muhammed Tan",
  "section": "5. Deploying Machine Learning Models",
  "question": "Error: failed to compute cache key: \"/model2.bin\" not found: not found"
}


In [64]:
result = search(course_piece['question'])


In [65]:
result

QueryResponse(points=[ScoredPoint(id=239, version=3, score=0.8333795, payload={'text': "ValueError: Path /Users/kt/.prefect/storage/44ccce0813ed4f24ab2d3783de7a9c3a does not exist.\nRemove ```cache_key_fn=task_input_hash ``` as it’s in argument in your function & run your flow again.\nNote: catche key is beneficial if you happen to run the code multiple times, it won't repeat the process which you have finished running in the previous run.  That means, if you have this ```cache_key``` in your initial run, this might cause the error.", 'section': "error: Error while reading table: trips_data_all.external_fhv_tripdata, error message: Parquet column 'DOlocationID' has type INT64 which does not match the target cpp_type DOUBLE.", 'course': 'data-engineering-zoomcamp'}, vector=None, shard_key=None, order_value=None)])

In [66]:
print(f"Question:\n{course_piece['question']}\n")
print("Top Retrieved Answer:\n{}\n".format(result.points[0].payload['text']))
print("Original Answer:\n{}".format(course_piece['text']))

Question:
Error: failed to compute cache key: "/model2.bin" not found: not found

Top Retrieved Answer:
ValueError: Path /Users/kt/.prefect/storage/44ccce0813ed4f24ab2d3783de7a9c3a does not exist.
Remove ```cache_key_fn=task_input_hash ``` as it’s in argument in your function & run your flow again.
Note: catche key is beneficial if you happen to run the code multiple times, it won't repeat the process which you have finished running in the previous run.  That means, if you have this ```cache_key``` in your initial run, this might cause the error.

Original Answer:
Initially, I did not assume there was a model2. I copied the original model1.bin and dv.bin. Then when I tried to load using
COPY ["model2.bin", "dv.bin", "./"]
then I got the error above in MINGW64 (git bash) on Windows.
The temporary solution I found was to use
COPY ["*", "./"]
which I assume combines all the files from the original docker image and the files in your working directory.
Added by Muhammed Tan


In [67]:
print(search("What if I submit homeworks late?").points[0].payload['text'])


No, late submissions are not allowed. But if the form is still not closed and it’s after the due date, you can still submit the homework. confirm your submission by the date-timestamp on the Course page.y
Older news:[source1] [source2]


In [68]:
client.create_payload_index(
    collection_name=collection_name,
    field_name="course",
    field_schema="keyword" # exact matching on string metadata fields
)

UpdateResult(operation_id=5, status=<UpdateStatus.COMPLETED: 'completed'>)

In [69]:
def search_in_course(query, course="mlops-zoomcamp", limit=1):

    results = client.query_points(
        collection_name=collection_name,
        query=models.Document( #embed the query text locally with "jinaai/jina-embeddings-v2-small-en"
            text=query,
            model=model_handle
        ),
        query_filter=models.Filter( # filter by course name
            must=[
                models.FieldCondition(
                    key="course",
                    match=models.MatchValue(value=course)
                )
            ]
        ),
        limit=limit, # top closest matches
        with_payload=True #to get metadata in the results
    )

    return results

In [70]:
print(search_in_course("What if I submit homeworks late?", "mlops-zoomcamp").points[0].payload['text'])


Please choose the closest one to your answer. Also do not post your answer in the course slack channel.
